In [1]:
import h5py
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gdp
from shapely.affinity import translate

import cartopy.crs as ccrs
import cartopy

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
%matplotlib inline

In [2]:
# ASI PRISMA and EO-1 HYPERION
df = pd.DataFrame(
    {'Probe': ['Land', 'Water', 'Wood'],
     'Color': ['#de6b00','#1f78b4','#3FB54B'],
     'Latitude': [-8.61482,-8.7912,-8.7040],
     'Longitude': [116.86549,116.7894,116.8355]})
gdf = gdp.GeoDataFrame(df, crs="EPSG:4326", geometry=gdp.points_from_xy(df.Longitude, df.Latitude))
# sort by label if needed
gdf = gdf.sort_values(by='Probe')
# save to file
gdf.to_file("probs.json", driver="GeoJSON")
gdf

,Probe,Color,Latitude,Longitude,geometry
0,Land,#de6b00,-8.61482,116.86549,POINT (116.86549 -8.61482)
1,Water,#1f78b4,-8.79120,116.78940,POINT (116.78940 -8.79120)
2,Wood,#3FB54B,-8.70400,116.83550,POINT (116.83550 -8.70400)


## Define EO-1 HYPERION Metadata

In [3]:
# See EO1H1150662006130110PY.hdr file in EO1H1150662006130110PY_1R.ZIP product 
wavelength = [
 355.59,  365.76,  375.94,  386.11,  396.29,  406.46,  416.64,  426.82,
 436.99,  447.17,  457.34,  467.52,  477.69,  487.87,  498.04,  508.22,
 518.39,  528.57,  538.74,  548.92,  559.09,  569.27,  579.45,  589.62,
 599.80,  609.97,  620.15,  630.32,  640.50,  650.67,  660.85,  671.02,
 681.20,  691.37,  701.55,  711.72,  721.90,  732.07,  742.25,  752.43,
 762.60,  772.78,  782.95,  793.13,  803.30,  813.48,  823.65,  833.83,
 844.00,  854.18,  864.35,  874.53,  884.70,  894.88,  905.05,  915.23,
 925.41,  935.58,  945.76,  955.93,  966.11,  976.28,  986.46,  996.63,
1006.81, 1016.98, 1027.16, 1037.33, 1047.51, 1057.68,  851.92,  862.01,
 872.10,  882.19,  892.28,  902.36,  912.45,  922.54,  932.64,  942.73,
 952.82,  962.91,  972.99,  983.08,  993.17, 1003.30, 1013.30, 1023.40,
1033.49, 1043.59, 1053.69, 1063.79, 1073.89, 1083.99, 1094.09, 1104.19,
1114.19, 1124.28, 1134.38, 1144.48, 1154.58, 1164.68, 1174.77, 1184.87,
1194.97, 1205.07, 1215.17, 1225.17, 1235.27, 1245.36, 1255.46, 1265.56,
1275.66, 1285.76, 1295.86, 1305.96, 1316.05, 1326.05, 1336.15, 1346.25,
1356.35, 1366.45, 1376.55, 1386.65, 1396.74, 1406.84, 1416.94, 1426.94,
1437.04, 1447.14, 1457.23, 1467.33, 1477.43, 1487.53, 1497.63, 1507.73,
1517.83, 1527.92, 1537.92, 1548.02, 1558.12, 1568.22, 1578.32, 1588.42,
1598.51, 1608.61, 1618.71, 1628.81, 1638.81, 1648.90, 1659.00, 1669.10,
1679.20, 1689.30, 1699.40, 1709.50, 1719.60, 1729.70, 1739.70, 1749.79,
1759.89, 1769.99, 1780.09, 1790.19, 1800.29, 1810.38, 1820.48, 1830.58,
1840.58, 1850.68, 1860.78, 1870.87, 1880.98, 1891.07, 1901.17, 1911.27,
1921.37, 1931.47, 1941.57, 1951.57, 1961.66, 1971.76, 1981.86, 1991.96,
2002.06, 2012.15, 2022.25, 2032.35, 2042.45, 2052.45, 2062.55, 2072.65,
2082.75, 2092.84, 2102.94, 2113.04, 2123.14, 2133.24, 2143.34, 2153.34,
2163.43, 2173.53, 2183.63, 2193.73, 2203.83, 2213.93, 2224.03, 2234.12,
2244.22, 2254.22, 2264.32, 2274.42, 2284.52, 2294.61, 2304.71, 2314.81,
2324.91, 2335.01, 2345.11, 2355.21, 2365.20, 2375.30, 2385.40, 2395.50,
2405.60, 2415.70, 2425.80, 2435.89, 2445.99, 2456.09, 2466.09, 2476.19,
2486.29, 2496.39, 2506.48, 2516.59, 2526.68, 2536.78, 2546.88, 2556.98,
2566.98, 2577.08]
fwhm = [
11.3871, 11.3871, 11.3871, 11.3871, 11.3871, 11.3871, 11.3871, 11.3871,
11.3871, 11.3871, 11.3871, 11.3871, 11.3871, 11.3784, 11.3538, 11.3133,
11.2580, 11.1907, 11.1119, 11.0245, 10.9321, 10.8368, 10.7407, 10.6482,
10.5607, 10.4823, 10.4147, 10.3595, 10.3188, 10.2942, 10.2856, 10.2980,
10.3349, 10.3909, 10.4592, 10.5322, 10.6004, 10.6562, 10.6933, 10.7058,
10.7276, 10.7907, 10.8833, 10.9938, 11.1044, 11.1980, 11.2600, 11.2824,
11.2822, 11.2816, 11.2809, 11.2797, 11.2782, 11.2771, 11.2765, 11.2756,
11.2754, 11.2754, 11.2754, 11.2754, 11.2754, 11.2754, 11.2754, 11.2754,
11.2754, 11.2754, 11.2754, 11.2754, 11.2754, 11.2754, 11.0457, 11.0457,
11.0457, 11.0457, 11.0457, 11.0457, 11.0457, 11.0457, 11.0457, 11.0457,
11.0457, 11.0457, 11.0457, 11.0457, 11.0457, 11.0457, 11.0457, 11.0451,
11.0423, 11.0372, 11.0302, 11.0218, 11.0122, 11.0013, 10.9871, 10.9732,
10.9572, 10.9418, 10.9248, 10.9065, 10.8884, 10.8696, 10.8513, 10.8335,
10.8154, 10.7979, 10.7822, 10.7663, 10.7520, 10.7385, 10.7270, 10.7174,
10.7091, 10.7022, 10.6970, 10.6946, 10.6937, 10.6949, 10.6996, 10.7058,
10.7163, 10.7283, 10.7437, 10.7612, 10.7807, 10.8034, 10.8267, 10.8534,
10.8818, 10.9110, 10.9422, 10.9743, 11.0074, 11.0414, 11.0759, 11.1108,
11.1461, 11.1811, 11.2156, 11.2496, 11.2826, 11.3146, 11.3460, 11.3753,
11.4037, 11.4302, 11.4538, 11.4760, 11.4958, 11.5133, 11.5286, 11.5404,
11.5505, 11.5580, 11.5621, 11.5634, 11.5617, 11.5563, 11.5477, 11.5346,
11.5193, 11.5002, 11.4789, 11.4548, 11.4279, 11.3994, 11.3688, 11.3366,
11.3036, 11.2696, 11.2363, 11.2007, 11.1666, 11.1333, 11.1018, 11.0714,
11.0424, 11.0155, 10.9912, 10.9698, 10.9508, 10.9355, 10.9230, 10.9139,
10.9083, 10.9069, 10.9057, 10.9013, 10.8951, 10.8854, 10.8740, 10.8591,
10.8429, 10.8242, 10.8039, 10.7820, 10.7592, 10.7342, 10.7092, 10.6834,
10.6572, 10.6312, 10.6052, 10.5803, 10.5560, 10.5328, 10.5101, 10.4904,
10.4722, 10.4552, 10.4408, 10.4285, 10.4197, 10.4129, 10.4088, 10.4077,
10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077,
10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077,
10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077, 10.4077,
10.4077, 10.4077]

## Read EO-1 HYPERION Multi-band GeoTiff Dataset (WGS84)

Use gdalbuildvrt tool to produce it from provided multiple GeoTiff files

In [4]:
da = xr.open_rasterio('EO1H1150662006130110PY_1T.tif')
# scale
#da = da.values/2**14-1
da['band'] = wavelength
da['fwhm'] = xr.DataArray(fwhm,dims=['band'])
da

<xarray.DataArray (band: 242, y: 4370, x: 1207)>
[1276450780 values with dtype=int16]
Coordinates:
  * band     (band) float64 355.6 365.8 375.9 ... 2.557e+03 2.567e+03 2.577e+03
  * y        (y) float64 -7.983 -7.983 -7.983 -7.984 ... -9.168 -9.168 -9.168
  * x        (x) float64 116.7 116.7 116.7 116.7 ... 117.0 117.0 117.0 117.0
    fwhm     (band) float64 11.39 11.39 11.39 11.39 ... 10.41 10.41 10.41 10.41
Attributes:
    transform:   (0.0002713732972756425, 0.0, 116.68415258079506, 0.0, -0.000...
    crs:         +init=epsg:4326
    res:         (0.0002713732972756425, 0.0002713732972756425)
    is_tiled:    0
    nodatavals:  (nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,...
    scales:      (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...
    offsets:     (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...

In [5]:
# convert probs to the specified coordinate system
#gdf = gdf.to_crs("EPSG:32750")
#gdf

In [6]:
# bands count
da.band.shape[0]

242

## EO-1 HYPERION Bands 30m

In [ ]:
plt.figure(figsize=(16,6))

delta = 0.005

wavelength = 721.9
band_vnir = da.sel(band=wavelength)

ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax1.coastlines(color='white')
ax1.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='k')
ax1.add_feature(cartopy.feature.OCEAN, zorder=0, edgecolor='lightblue')
ax1.set_global()
band_vnir[::5,::5].plot.pcolormesh(ax=ax1, transform=ccrs.PlateCarree(), cmap="Greys_r", add_colorbar=False)
ax1.set_xlim([116.6,117.1])
ax1.set_ylim([-8.9,-8.5])
ax1.set_title('EO-1 HYPERION VNIR 30m %.1f nm' % wavelength,fontsize=24)
# Probs
gdf.apply(lambda x: ax1.annotate(xy=translate(x.loc['geometry'],delta).coords[0],
                                 s=x.Probe, color=x.Color, fontsize=14), axis=1)
gdf.apply(lambda x: ax1.scatter(*x.loc['geometry'].coords[0], c=x.Color), axis=1)

wavelength = 1144.48
band_swir = da.sel(band=wavelength)

ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
ax2.coastlines(color='white')
ax2.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='k')
ax2.add_feature(cartopy.feature.OCEAN, zorder=0, edgecolor='lightblue')
ax2.set_global()
band_swir[::5,::5].plot.pcolormesh(ax=ax2, transform=ccrs.PlateCarree(),
                                             cmap="Greys_r", add_colorbar=False)
ax2.set_xlim([116.6,117.1])
ax2.set_ylim([-8.9,-8.5])
ax2.set_title('EO-1 HYPERION SWIR 30m %.1f nm' % wavelength,fontsize=24)
# Probs
gdf.apply(lambda x: ax2.annotate(xy=translate(x.loc['geometry'],delta).coords[0],
                                 s=x.Probe, color=x.Color, fontsize=14), axis=1)
gdf.apply(lambda x: ax2.scatter(*x.loc['geometry'].coords[0], c=x.Color), axis=1)

plt.savefig('EO-1 HYPERION VNIR and SWIR Bands.jpg', dpi=150, bbox_inches='tight')
plt.show()

## EO-1 HYPERION Spectrogram of Probe Locations

In [ ]:
plt.figure(figsize=(16,5.5))
plt.title('EO-1 HYPERION Spectrogram %d Bands (%.0fnm - %.0fnm)' %
         (da.band.shape[0],float(da.band.min()),float(da.band.max())),fontsize=24)
ax = plt.gca()

# for manual legend
patches = []
delta = 1e-3

# plot probes
for (idx,row) in gdf.iterrows():
    #print (row.Probe)
    
    probe = row.Probe
    coords = row.geometry.coords[0]
    color = row.Color

    patches.append(mpatches.Patch(color=color, label=probe))
    da_bands_subset = da.sel(x=slice(coords[0]-delta,coords[0]+delta),y=slice(coords[1]+delta,coords[1]-delta))

    bands_subset = da_bands_subset.to_dataframe(name='value').reset_index().dropna()

    ax = plt.gca()
    for key, grp in bands_subset.groupby(['x','y']):
        grp.plot(ax=ax, kind='line', x='band', y='value', color=color,lw=0.5,alpha=0.8)
    ax.get_legend().remove()

plt.legend(handles=patches, loc='upper right', fontsize=16)

plt.xlabel("Wavelength [nm]",size=16)
plt.ylabel("Radiance",size=16)
plt.xlim([400,2400])
plt.savefig('EO-1 HYPERION Spectrogram VNIR and SWIR Bands.jpg', dpi=150, bbox_inches='tight')
plt.show()